Connect to python API

In [27]:
%matplotlib inline

In [174]:
import json
config = {}
with open('config.json') as data_file:    
    config = json.load(data_file)


FileNotFoundError: [Errno 2] No such file or directory: 'config.json'

In [175]:
import twitter

# XXX: Go to http://dev.twitter.com/apps/new to create an app and get values
# for these credentials, which you'll need to provide in place of these
# empty string values that are defined as placeholders.
# See https://dev.twitter.com/docs/auth/oauth for more information 
# on Twitter's OAuth implementation.

CONSUMER_KEY = config['twitter']['CONSUMER_KEY']
CONSUMER_SECRET = config['twitter']['CONSUMER_SECRET']
OAUTH_TOKEN = config['twitter']['OAUTH_TOKEN']
OAUTH_TOKEN_SECRET = config['twitter']['OAUTH_TOKEN_SECRET']


auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                           CONSUMER_KEY, CONSUMER_SECRET)

twitter_api = twitter.Twitter(auth=auth)

# Nothing to see by displaying twitter_api except that it's now a
# defined variable

KeyError: 'twitter'

Get US and World Trends

In [ ]:
# The Yahoo! Where On Earth ID for the entire world is 1.
# See https://dev.twitter.com/docs/api/1.1/get/trends/place and
# http://developer.yahoo.com/geo/geoplanet/

WORLD_WOE_ID = 1
US_WOE_ID = 23424977
NYS_WOE_ID = 2459115
TX_WOE_ID = 2347602
UK_WOE_ID = 23424975


# Prefix ID with the underscore for query string parameterization.
# Without the underscore, the twitter package appends the ID value
# to the URL itself as a special case keyword argument.
trends = []

trends.append(twitter_api.trends.place(_id=US_WOE_ID)) #twitter_api.trends.place(_id=WORLD_WOE_ID)
trends.append(twitter_api.trends.place(_id=UK_WOE_ID)) #twitter_api.trends.place(_id=US_WOE_ID)

In [ ]:
import json

#print json.dumps(trends[0], indent=1)
#print
#print json.dumps(trends[1], indent=1)

In [ ]:
trend_set = []

trend_set.append(set([trend['name'] 
                        for trend in trends[0][0]['trends']]))

trend_set.append(set([trend['name'] 
                     for trend in trends[1][0]['trends']]))

common_trends = trend_set[0].intersection(trend_set[1])

trends_subtract = trend_set[1] - trend_set[0]

print ("Trend 1: \n" , trend_set[0])
print ("Trend 2: \n" , trend_set[1])

print ("\n{} common trends: \n".format(len(common_trends)), common_trends)

In [ ]:
queries = list(trend_set[0]) + list(trend_set[1])
print (len(queries))

In [ ]:
# Import unquote to prevent url encoding errors in next_results
#from urllib import unquote
from urllib.parse import unquote

# XXX: Set this variable to a trending topic, 
# or anything else for that matter. The example query below
# was a trending topic when this content was being developed
# and is used throughout the remainder of this chapter.

#queries = list(common_trends)



count = 500

trend_tweets = {}

# See https://dev.twitter.com/docs/api/1.1/get/search/tweets

for q in queries:
    print ("fetching {}".format(q))
    
    search_results = twitter_api.search.tweets(q=q, count=count)

    statuses = search_results['statuses']


    numberOfTweets = 100
    fetchIterations = int(numberOfTweets/count)

    for _ in range(fetchIterations):
        print ("Length of statuses {} for {}".format(len(statuses), q))
        try:
            next_results = search_results['search_metadata']['next_results']
        except e: # No more results when next_results doesn't exist
            break

        # Create a dictionary from next_results, which has the following form:
        # ?max_id=313519052523986943&q=NCAA&include_entities=1
        kwargs = dict([kv.split('=') for kv in unquote(next_results[1:]).split("&") ])    


        search_results = twitter_api.search.tweets(**kwargs)
        statuses += search_results['statuses']

    trend_tweets[q] = statuses


In [ ]:
import re
tweetText = trend_tweets[queries[0]][0]['text']
print (tweetText)
urlRegEx = "http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
noUrl = re.sub(urlRegEx, '', re.sub('RT @','@', tweetText))
print ('nourl', noUrl)

print (len(queries), len(trend_tweets.keys()))
print (trend_tweets[queries[0]][0]['lang'])
print (json.dumps(trend_tweets[queries[0]][0], indent=1))



In [ ]:
#print trend_tweets.keys()
import itertools

concatinatedText = {}
noUrlText = {}

urlRegEx = "http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
noUrl = re.sub(urlRegEx, '', tweetText)


for trend in trend_tweets.keys():
    tweet_list = trend_tweets[trend]
    #print tweet_list[0]
    #only english tweets
    concatinatedText[trend] = ''.join([ status['text'] if status['lang']=='en' else ''
                                for status in tweet_list])
    
    noUrlText[trend] = ''.join([ re.sub(urlRegEx, '', re.sub('RT @','@', re.sub('https[:/]*…', '', status['text']))) if status['lang']=='en' else ''
                                for status in tweet_list])

    print('==========================')
    print('TREND: ', trend)
    print('TEXT: ', noUrlText[trend])
    

print (len(trend_tweets.keys()))
print (len(noUrlText.values()))
#print (noUrlText.values())
#print concatinatedText['#SocialismChecklist']

In [ ]:
import math
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=2, stop_words = 'english',\
strip_accents = 'unicode', lowercase=True, ngram_range=(1,2),\
norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)
# X is a sparce martrix. each row containes the tfidfs for a trend.
X = vectorizer.fit_transform(noUrlText.values())
D = -(X * X.T).todense() # dot products
XX = X.todense()


ind = np.argsort(D)#[::-1][:4]

trendIndx = 1
print('comparing ', trends[0][0]['trends'][trendIndx])
for indx in np.nditer(ind[trendIndx]):
    trendGroup = int(math.floor(indx / 50))
    groupIndex = indx % 50
    trend =  trends[trendGroup][0]['trends'][groupIndex]
    if(trend['tweet_volume']):
        print(indx, 'british' if trendGroup==1 else 'US', trend['name'], trend['tweet_volume'])


        
countVectorizer = CountVectorizer(stop_words = 'english',\
strip_accents = 'unicode', lowercase=True)
CX = countVectorizer.fit_transform(noUrlText.values())




In [ ]:
import numpy as np

indices = np.argsort(vectorizer.idf_)[::-1]
features = vectorizer.get_feature_names()
top_n = 100
top_features = [features[i] for i in indices[:top_n]]
#print '\n Top {} words normalized by document frequency'.format(top_n)
#print top_features
#freqs = [(word, X.getcol(idx).sum()) for word, idx in vectorizer.vocabulary_.items()]
#sort from largest to smallest
#print sorted (freqs, key = lambda x: -x[1])

features = countVectorizer.get_feature_names()
top_n = 100
top_features = [features[i] for i in indices[:top_n]]
print ('\n Top {} features'.format(top_n))
print (top_features)

freqs = [(word, CX.getcol(idx).sum()) for word, idx in countVectorizer.vocabulary_.items()]
#sort from largest to smallest
print ('\n Top words by count')
top_words = sorted (freqs, key = lambda x: -x[1])
print (top_words[0:100])

In [ ]:
from sklearn.manifold import TSNE
model = TSNE(n_components=2, random_state=0)
np.set_printoptions(suppress=True)
coords = model.fit_transform(XX) 
print (coords)


In [ ]:
#import matplotlib.pyplot as plt

vis_x = coords[:, 0]
vis_y = coords[:, 1]


In [ ]:
import matplotlib.pyplot as plt

color = ['#FF0000'] * 50 + ['#0000FF'] * len(list(trends_subtract))


print (len(vis_x), len(color))

plt.scatter(vis_x, vis_y, color=color)

for i, txt in enumerate(queries):
    if i<len(vis_x): plt.annotate(txt, (vis_x[i],vis_y[i]))

fig = plt.gcf()
fig.set_size_inches(15.5, 15.5)

plt.show()

